![IFMG](https://images.even3.com.br/sBWZnWPFUBgLOGciSZc4G5ZQy7Q=/1100x440/smart/even3.blob.core.windows.net/banner/ARTEPARASITE3.993b3db9f908426e9833.png)

---

# Introdução ao Tensorflow - Parte 4

---

#### Professor: Felipe Reis

#### Data: 20-10-2021

---
### Importação de bibliotecas 

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import random
import time

#import plot_images
from xject_helper import xject_helper as helper

#tensorflow
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow import keras

---
## Base de Dados MNIST

O banco de dados MNIST contém dígitos manuscritos (escritos a mão).

Tem um conjunto de treinamento com 60.000 imagens e um conjunto de teste com 10.000 imagens. 

Os dígitos foram normalizados por tamanho e centralizados em uma imagem de tamanho fixo.

A base de dados foi criada por Yann LeCun em 1998.

A base MNIST está disponível em: [http://yann.lecun.com/exdb/mnist/](http://yann.lecun.com/exdb/mnist/)

![MNist](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)

Fonte: *Yann LeCun, Corinna Cortes, Christopher J.C. Burges. **Gradient-based learning applied to document recognition**. 1998. Proceedings of the IEEE, 86(11):2278-2324. Disponível em: http://yann.lecun.com/exdb/mnist/. Acesso em: 11 de outubro de 2021.*

In [ ]:
#download da base de dados
mnist = keras.datasets.mnist

#divisão em base de treinamento e testes (subdividido em imagens e labels)
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
#impressão da quantidade de imagens da base
print('Conjunto treino: {} amostras'.format(len(train_images)))
print('Conjunto teste: {} amostras'.format(len(test_images)))

In [ ]:
#definicao dos nomes das classes
#definição das classes 
class_names = ['Dígito 0', 'Dígito 1', 'Dígito 2', 'Dígito 3', 'Dígito 4', 
               'Dígito 5', 'Dígito 6', 'Dígito 7', 'Dígito 8', 'Dígito 9']

### Análise dos Dados

In [ ]:
helper.plot_images(train_images, train_labels, class_names, binary=True, random=False)

---
## Pré processamento

Podemos classificar o conjunto de dados utilizando os mesmos modelos utilizados no Notebook 1.

In [ ]:
#pré processamento
train_images = train_images / 255.0
test_images = test_images / 255.0

---
## Treinamento do Modelo

Os modelos des redes neurais abaixo foram baseado nas arquiteturas existentes no Notebook 1.

Para adequação ao conjunto CIFAR-10, a entrada foi alterada para suportar imagens de tamanho 32 $\times$ 32 pixels, com 3 canais de cores.

In [ ]:
def load_model_1():
    #modelo de rede sequencial
    return keras.Sequential([
        #transforma a image em um array de imagens de três dimensões (32 x 32 x 3)
        keras.layers.Flatten(input_shape=(28, 28), name='flatten_1'), 

        #Camada de dados totalmente conectadas, com ativação relu
        #A camada possui 128 nós (neurônios)
        keras.layers.Dense(128, activation='relu', name='dense_1'),

        #Camada de dados totalmente conectadas, com ativação softmax
        #A camada possui 10 nós (neurônios), correspondentes às probabilidades de cada classe (10 classes)
        keras.layers.Dense(10, activation='softmax', name='output')
    ])

In [ ]:
#gera modelo
model1 = load_model_1() #o modelo foi recarregado para melhor comparação dos otimizadores

#define o otimizador
adam = tf.keras.optimizers.Adam(learning_rate=0.001)

#compila o modelo
model1.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
history = model1.fit(train_images, train_labels, epochs=10)

# plota o gráfico
helper.plot_history_training(history, metrics=['accuracy'], legend=['Treinamento'], ylabel='Acurácia')

---
### Armazenamento e Recuperação de Pesos

Suponha que uma rede neural foi treinada durante um período de tempo. Após esse período, a rede será utilizada em uma aplicação. 

Neste cenário, é necessário o armazenamento de pesos da rede, de modo a evitar que sejam realizados novos treinamentos.

O armazenamento de pesos também pode ser usado em caso de interrupção do treinamento, permitindo que a rede não precise ser treinada desde o início novamente.

Esta seção contém os códigos necessários para armazenamento e recuperação de pesos.

#### Criação de Checkpoints

Para armazenar os pesos da rede, podemos criar checkpoints, que serão utilizados para salvar pesos com uma determinada frequência.

O código abaixo contém um checkpoint para uma rede neural.

In [ ]:
#define diretório onde o arquivo será salvo
checkpoint_path = 'models/model.hdf5'

#cria o checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

Após a criação do checkpoints, podemos adicioná-lo ao treinamento, dentro do método `model.fit`.

O parâmetro `callback` recebe uma método, que é invocado após o fim de uma época. É possível passar uma lista de callbacks para o treinamento da rede.

In [ ]:
#treina o modelo
history = model1.fit(train_images, train_labels, epochs=10, callbacks=[checkpoint])

Podemos alterar o checkpoint para salvar a execução somente se a rede melhorar seu desempenho em uma determinada métrica.

In [ ]:
#define diretório onde o arquivo será salvo
checkpoint_path = 'models/model.hdf5'

#cria o checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor='accuracy', 
                                                save_weights_only=True, save_best_only=True, verbose=1)

In [ ]:
#treina o modelo
history = model1.fit(train_images, train_labels, epochs=10, callbacks=[checkpoint])

---
### Recuperação de Pesos

Podemos recuperar os pesos antes do início do treinamento da rede. Para isso, devemos criar o modelo e em seguida carregar os pesos com auxílio do método `load_weights`.

In [ ]:
#define diretório onde o arquivo será salvo
checkpoint_path = 'models/model.hdf5'

#gera modelo
model1 = load_model_1() #o modelo foi recarregado para melhor comparação dos otimizadores

#carrega os pesos
model1.load_weights(checkpoint_path)

Após carregar os pesos, podemos executar o treinamento da rede.

In [ ]:
#define o otimizador
adam = tf.keras.optimizers.Adam(learning_rate=0.001)

#compila o modelo
model1.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
history = model1.fit(train_images, train_labels, epochs=10)

---
## Transfer Learning

*Transfer Learning* ou Transferência de Aprendizado corresponde à tarefa de aproveitar o conhecimento obtido em uma tarefa e aplicá-lo à outra.

Em redes neurais, esse conceito é associado à tarefa de reaproveitamento de pesos em tarefas diferentes.

#### *Transfer Learning* MNIST para Fashion MNIST

Suponha que uma rede foi treinada na base de dados MNIST. Após o treinamento, deseja-se que a rede seja utilizada na base Fashion MNIST.

As bases de dados são distintas, porém o conhecimento obtido pela rede na base MNIST pode ser utilizado para auxiliar no treinamento da base Fashion MNIST. Devido a características da rede, algumas *features*, principalmente das camadas iniciais da rede podem ser semelhantes. 

Aproveitar os pesos de uma rede em outra pode diminuir o número de épocas necessárias para treinamento. No entanto, para alguns modelos, o uso de pesos diferentes pode não causar nenhuma melhoria ou, até mesmo, piorar o desempenho da rede neural.

In [ ]:
#download da base de dados
fashion_mnist = keras.datasets.fashion_mnist

#divisão em base de treinamento e testes (subdividido em imagens e labels)
(fash_tr_images, fash_tr_labels), (fash_te_images, fash_te_labels) = fashion_mnist.load_data()

In [ ]:
#pré processamento
fash_tr_images = fash_tr_images / 255.0
fash_te_images = fash_te_images / 255.0

#### Treinamento da rede com Transfer Learning

In [ ]:
#define diretório onde o arquivo será salvo
checkpoint_path = 'models/model.hdf5'

#gera modelo
model1 = load_model_1() #o modelo foi recarregado para melhor comparação dos otimizadores

#carrega os pesos da rede MNIST
model1.load_weights(checkpoint_path)

#define o otimizador
adam = tf.keras.optimizers.Adam(learning_rate=0.001)

#compila o modelo
model1.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
history = model1.fit(fash_tr_images, fash_tr_labels, epochs=10)

---
### Transfer Learning entre Redes Diferentes

Podemos utilizar os pesos de uma rede em uma arquitetura de rede diferente (desde que semelhante).

Como indicado no exemplo anterior, esta tarefa também tem como objetivo diminuir o tempo de treinamento de uma rede neural.

### Base de Dados MNIST

Na tarefa atual, carregaremos um novo modelo de dados, contendo mais nós, semelhante ao recomendado no Notebook 1.

In [ ]:
def load_model_2():
    #modelo de rede sequencial
    return keras.Sequential([
        #transforma a image em um array de imagens de três dimensões (32 x 32 x 3)
        keras.layers.Flatten(input_shape=(28, 28), name='flatten_1'), 

        #Camada de dados totalmente conectadas, com ativação relu
        #A camada possui 128 nós (neurônios)
        keras.layers.Dense(128, activation='relu', name='dense_1'),
        keras.layers.Dense(128,  activation='relu', name='dense_2'),
        keras.layers.Dense(128,  activation='relu', name='dense_3'),

        #Camada de dados totalmente conectadas, com ativação softmax
        #A camada possui 10 nós (neurônios), correspondentes às probabilidades de cada classe (10 classes)
        keras.layers.Dense(10, activation='softmax', name='output')
    ])

Utilizaremos os pesos da rede 1.

É importante salientar que essa operação somente é valida caso as redes neurais sejam compatíveis. 

Em caso de arquiteturas de rede muito distintas, tal processo não irá funcionar adequadamente.

In [ ]:
#define diretório onde o arquivo será salvo
checkpoint_path = 'models/model.hdf5'

#gera modelo
model2 = load_model_2() #o modelo foi recarregado para melhor comparação dos otimizadores

#carrega os pesos da rede MNIST
model2.load_weights(checkpoint_path, by_name=True)

#define o otimizador
adam = tf.keras.optimizers.Adam(learning_rate=0.001)

#compila o modelo
model2.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
history = model2.fit(train_images, train_labels, epochs=10)

#### Treinamento somente de algumas camadas da rede

Se fazemos o *transfer learning* para uma mesma arquitetura de rede neural, pode não ser necessário o treinamento de todas as camadas da rede. 

Em redes diferentes, podemos apenas treinar as novas camadas. Esse procedimento reduz o tempo de treinamento, uma vez que os pesos dessas camadas estarão congelados. Caso queira, os pesos podem ser descongelados em um segundo momento, para refinamento de valores.

O parâmetro que indica que uma camada deve ser treinada é o `trainable`. Por padrão, esse parâmetro é verdadeiro.

In [ ]:
def load_model_3():
    #modelo de rede sequencial
    return keras.Sequential([
        #transforma a image em um array de imagens de três dimensões (32 x 32 x 3)
        keras.layers.Flatten(input_shape=(28, 28), name='flatten_1', trainable=False), 

        #Camada de dados totalmente conectadas, com ativação relu
        #A camada possui 128 nós (neurônios)
        keras.layers.Dense(128, activation='relu', name='dense_1', trainable=False),
        keras.layers.Dense(128,  activation='relu', name='dense_2'),
        keras.layers.Dense(128,  activation='relu', name='dense_3'),

        #Camada de dados totalmente conectadas, com ativação softmax
        #A camada possui 10 nós (neurônios), correspondentes às probabilidades de cada classe (10 classes)
        keras.layers.Dense(10, activation='softmax', name='output')
    ])

In [ ]:
#define diretório onde o arquivo será salvo
checkpoint_path = 'models/model.hdf5'

#gera modelo
model3 = load_model_3() #o modelo foi recarregado para melhor comparação dos otimizadores

#carrega os pesos da rede MNIST
model3.load_weights(checkpoint_path, by_name=True)

#define o otimizador
adam = tf.keras.optimizers.Adam(learning_rate=0.001)

#compila o modelo
model3.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
history = model3.fit(train_images, train_labels, epochs=10)

----
----

## Tarefas 

### Tarefa 1 (40 minutos)

Treine um conjunto de dados no modelo CIFAR-10 e salve os pesos. 

Faça o carregamento dos pesos para treinamento do conjunto CIFAR-100. Será necessário alterar a última camada da rede, devido ao número de classes na saída.

#### Atividades Sugeridas

- Utilize uma arquitetura de rede simples com mais de 3 camadas;
- Congele camadas ao treinar a rede CIFAR-100;
- Compare o treinamento da rede congelada com uma rede sem congelamento.
- Verifique se após descongelar os pesos, o desempenho do treinamento da rede melhora.

In [ ]:
#insira seu modelo aqui (arquitetura de rede)

In [ ]:
#compile seu modelo aqui

In [ ]:
#treine seu modelo aqui

In [ ]:
#imprima o gráfico de treinamento aqui

In [ ]:
#avalie seu modelo aqui